In [ ]:
import pandas as pd
import numpy as np
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import font_manager, rc
import scipy.cluster.hierarchy as spc
font_name = font_manager.FontProperties(fname="/usr/share/fonts/nanum/nanumGothic.ttf").get_name()
plt.rc('font', family=font_name)
plt.rc('style')
import warnings
warnings.filterwarnings(action='ignore') 
pd.options.display.float_format = '{:.2f}'.format
pd.set_option('max_colwidth', 1000)

In [ ]:
from IPython.display import display_html
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

In [ ]:
import os
file_list =os.listdir('../신세계/DATA/')

In [ ]:
for i in file_list:
    if i == ".ipynb_checkpoints":
        file_list.remove(i)
print(file_list)

In [ ]:
data_list = []
for i in file_list:
    print(i)
    data = pd.read_csv('../신세계/DATA/'+i, encoding='UTF-8',sep="\|\|",header=None,error_bad_lines=False)
    data_list.append(data)


In [ ]:
data = pd.concat(data_list ,axis=0)

In [ ]:
#data = pd.read_excel('data/alram_ds.xls',sheet_name=None, encoding='UTF-8')
#df_list = [ v for k,v in data.items()] 
#data = pd.concat(df_list ,axis=0)

## 1. 알람 데이터 로드 및 전처리

In [ ]:
data.columns = ["ALARM_LEVEL","ALARM_NAME","CTIME","DTIME","HOSTNAME","IP","PATH1","PATH2","PATH3","PATH4","PATH5","PATH6","PATH7","NAME","CONDITIONLOGTEXT","CURRENTALARMSTATUS"]

In [ ]:
#행렬 확인
data.shape

In [ ]:
#컬럼 이름 수정
data.columns = ['degree', 'alarm_name','time','clear_time','hostname','ip','group1','group2','group3','group4','group5','group6', 'group7','resource_name','condition','status']

In [ ]:
#추가 컬럼 생성
data.degree = data.degree.astype(str)
data['time'] = pd.to_datetime(data['time'], format='%Y-%m-%d %H:%M:%S')
data['clear_time'] = pd.to_datetime(data['clear_time'], format='%Y-%m-%d %H:%M:%S')
data['last_time'] = data['clear_time'] - data['time']
data['last_time'] = data['last_time'].dt.seconds
data['last_time'] = data['last_time'].fillna(0)
data['time_month'] = data['time'].dt.month
data['time_day'] = data['time'].dt.day
data['time_hour'] = data['time'].dt.hour
data['time_date']= data['time'].dt.strftime('%Y-%m-%d')
data['time_date_hour']= data['time'].dt.strftime('%Y-%m-%d %H')
ipaddr = data['ip'].str.split('.', expand=True)
ipaddr[3] = "0"
data['C_class']= ipaddr[0] + '.' + ipaddr[1] + '.' + ipaddr[2] + '.' + ipaddr[3]
ipaddr[3] = "0"
ipaddr[2] = "0"
data['B_class']= ipaddr[0] + '.' + ipaddr[1] + '.' + ipaddr[2] + '.' + ipaddr[3]

In [ ]:
data.degree = data.degree.replace({"1":"주의","2":"경고","3":"심각"})

In [ ]:
data.drop(812252,inplace=True)
data.drop(812251,inplace=True)

In [ ]:
data_log = data[data.alarm_name.isin(['PLog Info','PLog Warn','PLog Error','PLog Fatal','AMS_ERROR_POS','AMS_ERROR','Log Warning-Error - 복사본','Log Info','Log Warning-Error'])]

In [ ]:
data = data[~data.alarm_name.isin(['PLog Info','PLog Warn','PLog Error','PLog Fatal','AMS_ERROR_POS','AMS_ERROR','Log Warning-Error - 복사본','Log Info','Log Warning-Error'])]

In [ ]:
data['group'] = data.group1.fillna('')+'>'+data.group2.fillna('')+'>'+data.group3.fillna('')+'>'+data.group4.fillna('')+'>'+data.group5.fillna('')+'>'+data.group6.fillna('')+'>'+data.group7.fillna('')

In [ ]:
data['group'] = data.group.str.replace('>Root>','')

In [ ]:
data['group'] = data['group'].str.replace('Root>','')

In [ ]:
data['group'] = data['group'].str.replace(r'^SNMP 관제 그룹','02. Network>SNMP 관제 그룹')

In [ ]:
data['group'] = data['group'].str.replace(r'^PING관제그룹','02. Network>PING관제그룹')

In [ ]:
data['group'] = data['group'].str.replace('>>','')
data['group'] = data['group'].str.replace('>>>','')
data['group'] = data['group'].str.replace('>>>>','')

In [ ]:
data['group'] = data['group'].str.replace('>03. 보안','03. 보안')
data['group'] = data['group'].str.replace('>02. Network','02. Network')
data['group'] = data['group'].str.replace('>01. Server','01. Server')

In [ ]:
data['group'].str.split('>', expand=True)[0].unique()

In [ ]:
data['group1'] = data['group'].str.split('>', expand=True)[0]

data['group2'] = data['group'].str.split('>', expand=True)[1]

data['group3'] = data['group'].str.split('>', expand=True)[2]

data['group4'] = data['group'].str.split('>', expand=True)[3]

data['group5'] = data['group'].str.split('>', expand=True)[4]

data['group6'] = data['group'].str.split('>', expand=True)[5]

In [ ]:
server_data = data[data.group1=="01. Server"]

In [ ]:
network_data = data[data.group1=="02. Network"]

In [ ]:
server_data[server_data.group3=="[S2][DR][아,Uc] Polestar South DR 서버 (Active)"]

In [ ]:
rotation = 45

colors = ['orange','yellow','red']
f,ax=plt.subplots(1,1,figsize=(18,8))
sns.countplot('group2',hue='degree',data=server_data,order=server_data.group2.value_counts().iloc[:10].index, palette=sns.xkcd_palette(colors))
ax.set_title('group 별 심각도')
plt.xticks(rotation=rotation)
plt.show()

df = server_data.groupby(['group2','degree'])['time'].count().reindex().reset_index().sort_values('time',ascending=False)
df.columns = ['group1','degree','count']
df_1 = df[df['degree'] == '심각'].head(10)
df_2 = df[df['degree'] == '경고'].head(10)
df_3 = df[df['degree'] == '주의'].head(10)
display_side_by_side(df_1,df_2,df_3)

colors = ['orange','yellow','red']
f,ax=plt.subplots(1,1,figsize=(18,8))
sns.countplot('group3',hue='degree',data=server_data,order=server_data.group3.value_counts().iloc[:10].index, palette=sns.xkcd_palette(colors))
ax.set_title('group2 별 심각도')
plt.xticks(rotation=rotation)
plt.show()

df = server_data.groupby(['group3','degree'])['time'].count().reindex().reset_index().sort_values('time',ascending=False)
df.columns = ['group1','degree','count']
df_1 = df[df['degree'] == '심각'].head(10)
df_2 = df[df['degree'] == '경고'].head(10)
df_3 = df[df['degree'] == '주의'].head(10)
display_side_by_side(df_1,df_2,df_3)

colors = ['orange','yellow','red']
f,ax=plt.subplots(1,1,figsize=(18,8))
sns.countplot('group4',hue='degree',data=server_data,order=server_data.group4.value_counts().iloc[:10].index, palette=sns.xkcd_palette(colors))
ax.set_title('group3 별 심각도')
plt.xticks(rotation=rotation)
plt.show()

df = server_data.groupby(['group4','degree'])['time'].count().reindex().reset_index().sort_values('time',ascending=False)
df.columns = ['group1','degree','count']
df_1 = df[df['degree'] == '심각'].head(10)
df_2 = df[df['degree'] == '경고'].head(10)
df_3 = df[df['degree'] == '주의'].head(10)
display_side_by_side(df_1,df_2,df_3)

In [ ]:
rotation = 45

colors = ['orange','yellow','red']
f,ax=plt.subplots(1,1,figsize=(18,8))
sns.countplot('group2',hue='degree',data=network_data,order=network_data.group2.value_counts().iloc[:10].index, palette=sns.xkcd_palette(colors))
ax.set_title('group 별 심각도')
plt.xticks(rotation=rotation)
plt.show()

df = network_data.groupby(['group2','degree'])['time'].count().reindex().reset_index().sort_values('time',ascending=False)
df.columns = ['group1','degree','count']
df_1 = df[df['degree'] == '심각'].head(10)
df_2 = df[df['degree'] == '경고'].head(10)
df_3 = df[df['degree'] == '주의'].head(10)
display_side_by_side(df_1,df_2,df_3)

colors = ['orange','yellow','red']
f,ax=plt.subplots(1,1,figsize=(18,8))
sns.countplot('group3',hue='degree',data=network_data,order=network_data.group3.value_counts().iloc[:10].index, palette=sns.xkcd_palette(colors))
ax.set_title('group2 별 심각도')
plt.xticks(rotation=rotation)
plt.show()

df = network_data.groupby(['group3','degree'])['time'].count().reindex().reset_index().sort_values('time',ascending=False)
df.columns = ['group1','degree','count']
df_1 = df[df['degree'] == '심각'].head(10)
df_2 = df[df['degree'] == '경고'].head(10)
df_3 = df[df['degree'] == '주의'].head(10)
display_side_by_side(df_1,df_2,df_3)

colors = ['orange','yellow','red']
f,ax=plt.subplots(1,1,figsize=(18,8))
sns.countplot('group4',hue='degree',data=network_data,order=network_data.group4.value_counts().iloc[:10].index, palette=sns.xkcd_palette(colors))
ax.set_title('group3 별 심각도')
plt.xticks(rotation=rotation)
plt.show()

df = network_data.groupby(['group4','degree'])['time'].count().reindex().reset_index().sort_values('time',ascending=False)
df.columns = ['group1','degree','count']
df_1 = df[df['degree'] == '심각'].head(10)
df_2 = df[df['degree'] == '경고'].head(10)
df_3 = df[df['degree'] == '주의'].head(10)
display_side_by_side(df_1,df_2,df_3)

colors = ['orange','yellow','red']
f,ax=plt.subplots(1,1,figsize=(18,8))
sns.countplot('group5',hue='degree',data=network_data,order=network_data.group5.value_counts().iloc[:10].index, palette=sns.xkcd_palette(colors))
ax.set_title('group4 별 심각도')
plt.xticks(rotation=rotation)
plt.show()

df = network_data.groupby(['group5','degree'])['time'].count().reindex().reset_index().sort_values('time',ascending=False)
df.columns = ['group1','degree','count']
df_1 = df[df['degree'] == '심각'].head(10)
df_2 = df[df['degree'] == '경고'].head(10)
df_3 = df[df['degree'] == '주의'].head(10)
display_side_by_side(df_1,df_2,df_3)

In [ ]:
len(data_log)

In [ ]:
#data[['group1','group2','group3','group4','group5','group6','group7']]
len(data[(data.group1=="SNMP 관제 그룹")|(data.group2=="SNMP 관제 그룹")|(data.group3=="SNMP 관제 그룹")|(data.group4=="SNMP 관제 그룹")|(data.group5=="SNMP 관제 그룹")|(data.group6=="SNMP 관제 그룹")|(data.group7=="SNMP 관제 그룹")])

In [ ]:
data[(data.hostname=="[S1][스,Nu] 전자영수증 DB #1")&(data.alarm_name=="EventLog Info")].sort_values('time')

In [ ]:
data[(data.hostname=="[S1][사,Bl] 중앙POS")].sort_values('time')

In [ ]:
def search_date_diff(search_time,search_time2,degree,types):
    df1 = data[(data.time_date==search_time)&(data.degree==degree)].groupby([types])['ip'].count().reset_index().reindex().sort_values('ip',ascending=False)
    df2 = data[(data.time_date==search_time2)&(data.degree==degree)].groupby([types])['ip'].count().reset_index().reindex().sort_values('ip',ascending=False)
    merge_df =pd.merge(df1, df2, on=types, how='left').head(10)
    merge_df.columns = [types,search_time,search_time2]
    display(merge_df)
    

In [ ]:
search_date_diff('2018-12-06','2018-12-08','경고','alarm_name')
search_date_diff('2018-12-06','2018-12-08','주의','alarm_name')
search_date_diff('2018-12-06','2018-12-08','심각','alarm_name')

In [ ]:
def search_date_diff_alram_type(search_time,search_time2,degree,types):
    df1 = data[(data.time_date==search_time)&(data.degree==degree)&(data.alarm_name==types)].groupby(['hostname'])['ip'].count().reset_index().reindex().sort_values('ip',ascending=False)
    df2 = data[(data.time_date==search_time2)&(data.degree==degree)&(data.alarm_name==types)].groupby(['hostname'])['ip'].count().reset_index().reindex().sort_values('ip',ascending=False)
    merge_df =pd.merge(df1, df2, on='hostname', how='left')
    merge_df.columns = ['hostname',search_time,search_time2]
    merge_df.fillna(0,inplace=True)
    merge_df['diff'] = abs(merge_df[search_time] - merge_df[search_time2])
    merge_df.sort_values('diff',ascending=False).to_excel('diff_{}_{}.xlsx'.format(degree,types),encoding='UTF-8')
    display(merge_df.sort_values('diff',ascending=False))
    

In [ ]:
#search_date_diff('2018-12-06','2018-12-08','경고','alarm_name')
search_date_diff_alram_type('2018-12-06','2018-12-08','주의','SNMP 응답')
search_date_diff_alram_type('2018-12-06','2018-12-08','경고','Port_Down')
search_date_diff_alram_type('2018-12-06','2018-12-08','심각','장비다운')

In [ ]:
def search_date(search_time,search_time2):
    print("총알람 수 : {}".format(len(data[data.time_date==search_time])))
    display_side_by_side(
    data[data.time_date==search_time].groupby(['alarm_name'])['ip'].count().reset_index().reindex().sort_values('ip',ascending=False).head(10),
    data[data.time_date==search_time].groupby(['hostname'])['ip'].count().reset_index().reindex().sort_values('ip',ascending=False).head(10))
    display_side_by_side(
    data[(data.time_date==search_time)&(data.degree=="주의")].groupby(['degree','alarm_name'])['ip'].count().reset_index().reindex().sort_values('ip',ascending=False).head(10),
    data[(data.time_date==search_time)&(data.degree=="경고")].groupby(['degree','alarm_name'])['ip'].count().reset_index().reindex().sort_values('ip',ascending=False).head(10),
    data[(data.time_date==search_time)&(data.degree=="심각")].groupby(['degree','alarm_name'])['ip'].count().reset_index().reindex().sort_values('ip',ascending=False).head(10))
    display_side_by_side(
    data[(data.time_date==search_time)&(data.degree=="주의")].groupby(['degree','alarm_name','hostname'])['ip'].count().reset_index().reindex().sort_values('ip',ascending=False).head(10),
    data[(data.time_date==search_time)&(data.degree=="경고")].groupby(['degree','alarm_name','hostname'])['ip'].count().reset_index().reindex().sort_values('ip',ascending=False).head(10),
    data[(data.time_date==search_time)&(data.degree=="심각")].groupby(['degree','alarm_name','hostname'])['ip'].count().reset_index().reindex().sort_values('ip',ascending=False).head(10))

In [ ]:
search_date('2018-12-06')

##데이터 월 정제용

data = data[(data.time_date >= '2018-04-01')&(data.time_date < '2019-04-01')]

import datetime

data['time'] = data['time'] - datetime.timedelta(days=90)

data['time_month'] = data['time'].dt.month

## 2. 데이터 분석

In [ ]:
#미복구 알람
data[data.clear_time.isnull()==True].groupby(['degree','alarm_name','hostname'])['ip'].count().reindex().reset_index().sort_values('ip',ascending=False)

### 노드 별 심각도 건수

In [ ]:
df = data.groupby(['hostname','degree'])['alarm_name'].count().reindex().reset_index()
alram_count = df.pivot_table(values='alarm_name', index=df.hostname, columns='degree', aggfunc='first',fill_value=0).reset_index()
alram_count['total_count'] = alram_count['경고'] + alram_count['주의'] + alram_count['심각']
alram_count.sort_values('total_count',ascending=False).head(20).total_count

### 등급별 건수

In [ ]:
df = data.groupby(['degree'])['alarm_name'].count().reindex().reset_index()
df['per'] = (df.alarm_name/df.alarm_name.sum())*100
df['per']= df['per'].round(2)
df.columns = ['degree', 'count' , 'per']
#fig = plt.figure(figsize=(15, 7))
#ax1 = fig.add_subplot(1, 2, 1)
#ax2 = fig.add_subplot(1, 2, 2)
#ax2.axis('off')
#ax2.axis('tight')
#ax1.pie(df.per, explode=[0.05,0.05,0.05],labels=df.degree, autopct='%1.1f%%',colors=['orange','red','yellow'],textprops={'fontsize': 20},
       #shadow=True, startangle=90)
#ax2.table(cellText=df.values, colLabels=df.columns, loc='center')
df

### 일별 등급 알람건수

In [ ]:
df = data.groupby(['time_date','degree'])['alarm_name'].count().reindex().reset_index()
df = df[df['degree'] == '심각']
df.sort_values('alarm_name',ascending=False).head()

In [ ]:
df = data.groupby(['time_date','degree'])['alarm_name'].count().reindex().reset_index()
df = df[df['degree'] == '주의']
df.sort_values('alarm_name',ascending=False).head()

In [ ]:
df = data.groupby(['time_date','degree'])['alarm_name'].count().reindex().reset_index()
df = df[df['degree'] == '경고']
df.sort_values('alarm_name',ascending=False).head()

### 노드별 발생 알람이름 건수

In [ ]:
df = data.groupby(['alarm_name'])['time'].count().reindex().reset_index()
#df = df.pivot_table(values='time', index=df.alarm_name, columns='alarm_name', aggfunc='first',fill_value=0).reset_index()
#df = df.set_index('alarm_name')
#df.reindex(columns='시스템명')
#df['total_count']= df.sum(axis=1)
df.sort_values('time',ascending=False).head(10)

In [ ]:
df = data.groupby(['hostname'])['time'].count().reindex().reset_index().sort_values('time')
df[df['time'] == 1]

In [ ]:
server_group = data[(data.group1=="01. Server")|(data.group2=="01. Server")|(data.group3=="01. Server")|(data.group4=="01. Server")|(data.group5=="01. Server")|(data.group6=="01. Server")|(data.group7=="01. Server")]

In [ ]:
len(server_group)

In [ ]:
display_side_by_side(server_group[server_group.degree=='심각'].groupby(['alarm_name','degree'])['ip'].count().reset_index().reindex().sort_values('ip',ascending=False).head(10),
                    server_group[server_group.degree=='주의'].groupby(['alarm_name','degree'])['ip'].count().reset_index().reindex().sort_values('ip',ascending=False).head(10),
                    server_group[server_group.degree=='경고'].groupby(['alarm_name','degree'])['ip'].count().reset_index().reindex().sort_values('ip',ascending=False).head(10))

In [ ]:
server_group.groupby(['hostname','alarm_name'])['ip'].count().reset_index().reindex().sort_values('ip',ascending=False).head(10)

In [ ]:
network_group = data[(data.group1=="02. Network")|(data.group2=="02. Network")|(data.group3=="02. Network")|(data.group4=="02. Network")|(data.group5=="02. Network")|(data.group6=="02. Network")|(data.group7=="02. Network")]

In [ ]:
len(network_group)

In [ ]:
display_side_by_side(network_group[network_group.degree=='심각'].groupby(['alarm_name','degree'])['ip'].count().reset_index().reindex().sort_values('ip',ascending=False).head(10),
                     network_group[network_group.degree=='경고'].groupby(['alarm_name','degree'])['ip'].count().reset_index().reindex().sort_values('ip',ascending=False).head(10),
                     network_group[network_group.degree=='주의'].groupby(['alarm_name','degree'])['ip'].count().reset_index().reindex().sort_values('ip',ascending=False).head(10))

In [ ]:
network_group.groupby(['hostname','alarm_name'])['ip'].count().reset_index().reindex().sort_values('ip',ascending=False).head(10)

In [ ]:
df1 = data.groupby(['group4','degree'])['time'].count().reset_index().reindex()
df1.columns = ['group','degree','count']
df2 = data.groupby(['group5','degree'])['time'].count().reset_index().reindex()
df2.columns = ['group','degree','count']
df3 = data.groupby(['group6','degree'])['time'].count().reset_index().reindex()
df3.columns = ['group','degree','count']

In [ ]:
df_result =pd.merge(pd.merge(df1,df2,on=['group','degree']),df3,on=['group','degree'])

In [ ]:
df_result['sum'] = pd.merge(pd.merge(df1,df2,on=['group','degree']),df3,on=['group','degree']).sum(axis=1)

In [ ]:
df_result.drop(['count_x','count_y','count'],axis=1,inplace=True)

In [ ]:
df_result.pivot_table(values='sum', index='group', columns=df_result.degree, aggfunc='first',fill_value=0).reset_index().to_excel('result_group.xlsx',encoding='UTF-8')

In [ ]:
df2 = data.groupby(['group5','degree'])['time'].count().reset_index().reindex()

In [ ]:
data.groupby(['group5','degree'])['time'].count().reset_index().reindex()

In [ ]:
data.groupby(['group6','degree'])['time'].count().reset_index().reindex()

### 

In [ ]:
rotation = 45

colors = ['orange','yellow','red']
f,ax=plt.subplots(1,1,figsize=(18,8))
sns.countplot('group4',hue='degree',data=data,order=data.group1.value_counts().iloc[:10].index, palette=sns.xkcd_palette(colors))
ax.set_title('group 별 심각도')
plt.xticks(rotation=rotation)
plt.show()
df = data.groupby(['group1','degree'])['time'].count().reindex().reset_index().sort_values('time',ascending=False)
df.columns = ['group1','degree','count']
df_1 = df[df['degree'] == '심각'].head(10)
df_2 = df[df['degree'] == '경고'].head(10)
df_3 = df[df['degree'] == '주의'].head(10)
display_side_by_side(df_1,df_2,df_3)

colors = ['orange','yellow','red']
f,ax=plt.subplots(1,1,figsize=(18,8))
sns.countplot('group5',hue='degree',data=data,order=data.group2.value_counts().iloc[:10].index, palette=sns.xkcd_palette(colors))
ax.set_title('group2 심각도')
plt.xticks(rotation=rotation)
plt.show()
df = data.groupby(['group2','degree'])['time'].count().reindex().reset_index().sort_values('time',ascending=False)
df.columns = ['group2','degree','count']
df_1 = df[df['degree'] == '심각'].head(10)
df_2 = df[df['degree'] == '경고'].head(10)
df_3 = df[df['degree'] == '주의'].head(10)
display_side_by_side(df_1,df_2,df_3)

colors = ['orange','yellow','red']
f,ax=plt.subplots(1,1,figsize=(18,8))
sns.countplot('group3',hue='degree',data=data,order=data.group3.value_counts().iloc[:10].index, palette=sns.xkcd_palette(colors))
ax.set_title('group3 심각도')
plt.xticks(rotation=rotation)
plt.tick_params(axis='both', which='minor', labelsize=0.2)
plt.show()
df = data.groupby(['group6','degree'])['time'].count().reindex().reset_index().sort_values('time',ascending=False)
df.columns = ['group6','degree','count']
df_1 = df[df['degree'] == '심각'].head(10)
df_2 = df[df['degree'] == '경고'].head(10)
df_3 = df[df['degree'] == '주의'].head(10)
display_side_by_side(df_1,df_2,df_3)

colors = ['orange','yellow','red']
f,ax=plt.subplots(1,1,figsize=(18,8))
sns.countplot('hostname',hue='degree',data=data,order=data.hostname.value_counts().iloc[:10].index, palette=sns.xkcd_palette(colors))
ax.set_title('시스템별 심각도')
plt.xticks(rotation=rotation)
plt.tick_params(axis='both', which='minor', labelsize=0.2)
plt.show()
df = data.groupby(['hostname','degree'])['time'].count().reindex().reset_index().sort_values('time',ascending=False)
df.columns = ['hostname','degree','count']
df_1 = df[df['degree'] == '심각'].head(10)
df_2 = df[df['degree'] == '경고'].head(10)
df_3 = df[df['degree'] == '주의'].head(10)
display_side_by_side(df_1,df_2,df_3)

colors = ['orange','yellow','red']
f,ax=plt.subplots(1,1,figsize=(18,8))
sns.countplot('alarm_name',hue='degree',data=data,order=data.alarm_name.value_counts().iloc[:10].index, palette=sns.xkcd_palette(colors))
ax.set_title('알람 이름 별 심각도')
plt.xticks(rotation=rotation)
plt.tick_params(axis='both', which='minor', labelsize=0.2)
plt.show()
df = data.groupby(['alarm_name','degree'])['time'].count().reindex().reset_index().sort_values('time',ascending=False)
df.columns = ['alarm_name','degree','count']
df_1 = df[df['degree'] == '심각'].head(10)
df_2 = df[df['degree'] == '경고'].head(10)
df_3 = df[df['degree'] == '주의'].head(10)
display_side_by_side(df_1,df_2,df_3)


colors = ['orange','yellow','red']
f,ax=plt.subplots(1,1,figsize=(18,8))
sns.countplot('C_class',hue='degree',data=data,order=data.C_class.value_counts().iloc[:10].index, palette=sns.xkcd_palette(colors))
ax.set_title('C_class')
plt.xticks(rotation=rotation)
plt.tick_params(axis='both', which='minor', labelsize=0.2)
plt.show()
df = data.groupby(['C_class','degree'])['time'].count().reindex().reset_index().sort_values('time',ascending=False)
df.columns = ['C_class','degree','count']
df_1 = df[df['degree'] == '심각'].head(10)
df_2 = df[df['degree'] == '경고'].head(10)
df_3 = df[df['degree'] == '주의'].head(10)
display_side_by_side(df_1,df_2,df_3)


colors = ['orange','yellow','red']
f,ax=plt.subplots(1,1,figsize=(18,8))
sns.countplot('B_class',hue='degree',data=data,order=data.B_class.value_counts().iloc[:10].index, palette=sns.xkcd_palette(colors))
ax.set_title('B_class')
plt.xticks(rotation=rotation)
plt.tick_params(axis='both', which='minor', labelsize=0.2)
plt.show()
df = data.groupby(['B_class','degree'])['time'].count().reindex().reset_index().sort_values('time',ascending=False)
df.columns = ['B_class','degree','count']
df_1 = df[df['degree'] == '심각'].head(10)
df_2 = df[df['degree'] == '경고'].head(10)
df_3 = df[df['degree'] == '주의'].head(10)
display_side_by_side(df_1,df_2,df_3)

### 3. time_date(특정일) time_hour(시간) time_weekday(요일) time_day(일) time_week(주) 별 알람 건수

In [ ]:
'''
df = data.groupby(['hostname','time_date'])['time'].count().reindex().reset_index()
df.columns =['hostname', 'time_date', 'total_count']
df = df.pivot_table(values='total_count', index=df['time_date'], columns=df['hostname'], aggfunc='first',fill_value=0)
df['total_count'] = df.sum(axis=1)
df = df.sort_values('total_count',ascending=False)
'''
df_hour = data.groupby(['time_hour','degree'])['time'].count().reindex().reset_index()
df_hour.columns = ['time_hour', 'degree' , 'total_count']

data['time_weekday'] = data['time'].dt.weekday
df_weekday = data.groupby(['time_weekday','degree'])['time'].count().reindex().reset_index()
df_weekday.columns = ['time_weekday', 'degree' , 'total_count']

df_days = data.groupby(['time_day','degree'])['time'].count().reindex().reset_index()
df_days.columns = ['time_day', 'degree' , 'total_count']

data['time_week'] = data['time'].dt.week
df_week = data.groupby(['time_week','degree'])['time'].count().reindex().reset_index()
df_week.columns = ['time_week', 'degree' , 'total_count']

In [ ]:
df_month = data.groupby(['time_month','degree'])['time'].count().reindex().reset_index()
df_month.columns = ['time_month', 'degree' , 'total_count']

In [ ]:
df = data.groupby(['time_date','degree'])['time'].count().reindex().reset_index()
df.columns = ['time_date', 'degree' , 'total_count']
display_side_by_side(df[df.degree.str.match('심각')==True].sort_values('total_count', ascending=False).head(),
        df[df.degree.str.match('주의')==True].sort_values('total_count', ascending=False).head(),
        df[df.degree.str.match('경고')==True].sort_values('total_count', ascending=False).head())

In [ ]:
display_side_by_side(df_hour[df_hour.degree.str.match('심각')==True].sort_values('total_count', ascending=False).head(),
df_hour[df_hour.degree.str.match('주의')==True].sort_values('total_count', ascending=False).head(),
df_hour[df_hour.degree.str.match('경고')==True].sort_values('total_count', ascending=False).head())

In [ ]:
display_side_by_side(df_days[df_days.degree.str.match('심각')==True].sort_values('total_count', ascending=False).head(),
df_days[df_days.degree.str.match('주의')==True].sort_values('total_count', ascending=False).head(),
df_days[df_days.degree.str.match('경고')==True].sort_values('total_count', ascending=False).head())

In [ ]:
display_side_by_side(df_weekday[df_weekday.degree.str.match('심각')==True].sort_values('total_count', ascending=False).head(),
df_weekday[df_weekday.degree.str.match('주의')==True].sort_values('total_count', ascending=False).head(),
df_weekday[df_weekday.degree.str.match('경고')==True].sort_values('total_count', ascending=False).head())


In [ ]:
display_side_by_side(df_week[df_week.degree.str.match('심각')==True].sort_values('total_count', ascending=False).head(),
df_week[df_week.degree.str.match('주의')==True].sort_values('total_count', ascending=False).head(),
df_week[df_week.degree.str.match('경고')==True].sort_values('total_count', ascending=False).head())


In [ ]:
display_side_by_side(df_month[df_month.degree.str.match('심각')==True].sort_values('total_count', ascending=False).head(),
df_month[df_month.degree.str.match('주의')==True].sort_values('total_count', ascending=False).head(),
df_month[df_month.degree.str.match('경고')==True].sort_values('total_count', ascending=False).head())


In [ ]:
#colors = ['orange','red','yellow']
#sns.set(style="ticks",rc={"lines.linewidth": 0.7})
sns.factorplot(data=df, x="time_date", y="total_count",hue='degree',legend_out=True,markers=['None','None','None'] ,size=3, aspect=4)
plt.xticks(rotation='vertical')
plt.title('일자별 알람 추이')

In [ ]:
sns.factorplot(data=df_hour, x="time_hour", y="total_count",hue='degree',legend_out=True, size=3, aspect=4)
plt.title('시간별 알람 추이')

sns.factorplot(data=df_days, x="time_day", y="total_count",hue='degree',legend_out=True, size=3, aspect=4)
plt.title('일자별 알람 추이(1일~31일)')
sns.factorplot(data=df_weekday, x="time_weekday", y="total_count",hue='degree',legend_out=True, size=3, aspect=4)
plt.title('요일별 알람 추이')
sns.factorplot(data=df_week, x="time_week", y="total_count",hue='degree',legend_out=True, size=3, aspect=4)
plt.title('주별 알람 추이')
sns.factorplot(data=df_month, x="time_month", y="total_count",hue='degree',legend_out=True, size=3, aspect=4)
plt.title('월별 알람 추이')

In [ ]:
sns.factorplot(data=df_month, x="time_month", y="total_count",hue='degree',legend_out=True, size=3, aspect=4)
plt.title('월별 알람 추이')

### 4. 특정 일에 발생한 알람 확인을 위한 코드

In [ ]:
df = data.groupby(['time_date','degree','alarm_name','hostname','condition'])['ip'].count().reindex().reset_index()
df.columns = ['time_date', 'degree' ,'alarm_name','hostname','condition', 'total_count']

display(df[df['degree'] == '심각'].sort_values('total_count', ascending=False).head(30))
display(df[df['degree'] == '경고'].sort_values('total_count', ascending=False).head(30))
display(df[df['degree'] == '주의'].sort_values('total_count', ascending=False).head(30))

df = data.groupby(['time_date_hour','degree'])['ip'].count().reindex().reset_index()
df.columns = ['time_date_hour', 'degree' , 'total_count']

sns.factorplot(data=df, x="time_date_hour", y="total_count",hue='degree',legend_out=True, size=20, aspect=4 ,)
plt.xticks(rotation='vertical')
plt.title('시간별 알람 추이')

display(df[df['degree'] == '심각'].sort_values('total_count', ascending=False).head())
index_time = df[df['degree'] == '심각'].sort_values('total_count', ascending=False).head().time_date
for time in index_time:
   df[(df['degree'] == '심각')&(df['time'].astype('str').isin(time))]).sort_values('total_count', ascending=False).head(10))

((df['degree'] == '심각')&(df['time'].astype('str').isin(time)))
#display(df[df['degree'] == '경고'].sort_values('total_count', ascending=False).head())
#display(df[df['degree'] == '주의'].sort_values('total_count', ascending=False).head())

In [ ]:
df = data.groupby(['time','degree','alarm_name','hostname','condition'])['ip'].count().reindex().reset_index()
df.columns = ['time', 'degree' ,'alarm_name','hostname','condition', 'total_count']

display_side_by_side(df[df['degree'] == '심각'].sort_values('total_count', ascending=False).head(10))
display_side_by_side(df[df['degree'] == '경고'].sort_values('total_count', ascending=False).head(10))
display_side_by_side(df[df['degree'] == '주의'].sort_values('total_count', ascending=False).head(10))

### 5. 발생 알람 건수 분포 히스토그램

In [ ]:
df = data.groupby(['hostname'])['alarm_name'].count().reindex().reset_index()
sns.distplot( df["alarm_name"], bins=100 )

In [ ]:
df.sort_values('alarm_name',ascending=False)

### 6. 노드 단위로 발생 알람 건수를 groupby 해서 75% (3분위 수) 120건 미만으로 발생한 노드는 제외함 (Cluster Step 1)

display(df[(df['degree'] == '심각')&((df['time'] >= '2018-07-21')&(df['time'] <= '2018-07-22'))].sort_values('total_count', ascending=False).head(100))


In [ ]:
data_df = data.groupby(['hostname'])['time'].count().reindex().reset_index()
data_df.sort_values('time',ascending=False)
len(data_df[(data_df.time >= 1)&(data_df.time < 1000)])

In [ ]:
len(data.hostname.unique())

In [ ]:
data_df = data.groupby(['hostname'])['time'].count().reindex().reset_index()

In [ ]:
data_df = data_df.rename(columns={'time': 'count'})

In [ ]:
data_df.describe()

In [ ]:
data_df = data_df[data_df['count'] > 1000]

In [ ]:
node_names = data_df.sort_values('count',ascending=False).hostname.values

In [ ]:
c_nodes = data[data['hostname'].isin(node_names)==True]

In [ ]:
data_df_date = c_nodes.groupby(['time_date','hostname'])['time'].count().reindex().reset_index()
data_df_date = data_df_date.pivot_table(values='time',columns='hostname',index=data_df_date['time_date'],aggfunc='first',fill_value=0)

### 7. 노드별로 1달 동안 일별로 발생한 알람 건수를 가지고 각 노드들 끼리 알람 건수에 대한 상관도 측정 (Cluster Step 2)

for node in data_df_date.reindex(sorted(data_df_date.columns), axis=1).columns:
    print('>>>>>>>> '+node)
    fig = plt.figure(figsize=(15, 7))
    ax1 = fig.add_subplot(2, 1, 1)
    ax2 = fig.add_subplot(2, 1, 2)
    plt.subplots_adjust(top=0.5)
    ax2.axis('off')
    ax2.axis('tight')
    c_nodes = data[data['system_name']==node]
    dataset = c_nodes.groupby(['time_date','alarm_name'])['time'].count().reindex().reset_index().sort_values('time', ascending=False).head(10)
    ax2.table(cellText=dataset.values, colLabels=dataset.columns, loc='bottom')
    plt.setp(ax1.get_xticklabels(), rotation=90)
    sns.pointplot(x=data_df_date.index, y=node, data=data_df_date,ax=ax1,rotation=90)
    plt.show()
    plt.clf()

In [ ]:
corrmat = data_df_date.corr()

### 8. 상관분석 결과 (Cluster Step 3)

In [ ]:
corrmat[corrmat.index == '']

In [ ]:
f, ax = plt.subplots(figsize=(100,100))
sns_plot = sns.heatmap(corrmat, ax=ax, cmap="YlGnBu", linewidths=0.1)
sns_plot.figure.savefig("output.png")

### 9. 상관분석 결과 상관도가 90% 이상인 노드들끼리 클러스터링 (Cluster Result)

In [ ]:
clusters = []
for node in corrmat.columns:
    clusters.append(corrmat[corrmat[node] >= 0.9].index)
clusters_new = []
for cluster in clusters:
    if len(cluster) > 1:
        clusters_new.append(list(cluster))
clusters_new = list(set([tuple(set(node)) for node in clusters_new]))

dictt = {}
for i in range(0,len(clusters_new)):
    dictt.update({'cluster{}'.format(i) : list(clusters_new[i])})

#pd.concat([pd.Series(clusters_new[0],name='cluster0'),pd.Series(clusters_new[1],name='cluster1')], axis=1)
#for i in range(2,len(clusters_new)):
#    df = pd.concat([df,pd.Series(clusters_new[i],name='cluster{}'.format(i))], axis=1)
#df.fillna('',inplace=True)

In [ ]:
len(clusters_new)

In [ ]:
clusters_new = [['']]

### 10. 클러스터에 속한 각각의 노드의 한달 알람 발생 추의와 주요 알람 지표를 출력 (Visualization for Cluster Result)

In [ ]:
for cluster in clusters_new:
    print('######### '+ str(cluster))
    for node in cluster:
        print('>>>>>>>> '+node)
        fig = plt.figure(figsize=(15, 7))
        ax1 = fig.add_subplot(2, 1, 1)
        ax2 = fig.add_subplot(2, 1, 2)
        plt.subplots_adjust(top=0.5)
        ax2.axis('off')
        ax2.axis('tight')
        c_nodes = data[data['hostname']==node]
        dataset = c_nodes.groupby(['time_date','alarm_name'])['time'].count().reindex().reset_index().sort_values('time', ascending=False).head(10)
        ax2.table(cellText=dataset.values, colLabels=dataset.columns, loc='bottom')
        plt.setp(ax1.get_xticklabels(), rotation=90)
        sns.pointplot(x=data_df_date.index, y=node, data=data_df_date,ax=ax1,rotation=90)
        plt.show()
        plt.clf()

### 10-1 클러스터 별로 한달 알람 발생 추의와 주요 알람 지표를 출력

In [ ]:
data_df_date = data_df_date.reset_index()

In [ ]:
#plt.legend(fontsize='x-large', title_fontsize='40')
#sns.set(font_scale=1.5,style='ticks',font=font_name,rc={"lines.linewidth": 0.7})
i = 0
for cluster in clusters_new:
    print('>>>>>>>>>cluster{}'.format(i))
    test_data = pd.melt(data_df_date, id_vars=['time_date'],value_vars=cluster)
    fig = plt.figure(figsize=(15, 7))
    ax = sns.pointplot(x="time_date", y="value",hue=test_data.hostname, data=test_data)
    plt.xticks(rotation='vertical')
    plt.show()
    plt.clf()
    i += 1

In [ ]:
i = 0
c_nodes = data[data['hostname'].isin(clusters_new[17])]
c_nodes = c_nodes.groupby(['time_date','alarm_name','hostname'])['time'].count().reindex().reset_index().sort_values('time_date', ascending=True)
c_nodes = c_nodes.groupby(['alarm_name','hostname'])['time'].sum().reindex().reset_index().sort_values('time', ascending=False)
c_nodes = c_nodes[c_nodes.time > 50]
display_side_by_side(c_nodes)

In [ ]:
i = 0
for cluster in clusters_new:
    print('>>>>>>>cluster{}\n{}'.format(i,cluster))
    c_nodes = data[data['hostname'].isin(cluster)]
    c_nodes = c_nodes.groupby(['time_date','alarm_name','hostname'])['time'].count().reindex().reset_index().sort_values('time_date', ascending=True)
    c_nodes = c_nodes.groupby(['alarm_name','hostname'])['time'].sum().reindex().reset_index().sort_values('time', ascending=False)
    c_nodes = c_nodes[c_nodes.time > 100]
    display_side_by_side(c_nodes)
    i += 1

data[(data.system_name == 'tsgwap') & (data.time_date=='2018-08-03')]

pd.set_option('display.max_colwidth', -1)
data[(data.system_name == 'tsgwap') & (data.time_date=='2018-08-03')].condition

### 11-1. 클러스터 0번에 속한 노드의 주요 지표 디스크 Top I/O 처리율에 대한 노드별 심각도 알람 건수 추이 출력
> (클러스터 0번이 첫번재 결과라 선택 함) - cluster0 ('HFEU-TREC234', 'HKCC-TREC231', 'HKCC-TREC236')
> 날짜별 현황

In [ ]:
df = data[(data.alarm_name == 'CPU Wait Process 수') & (data['hostname'].isin(clusters_new[1]))]
#df = df[df['system_name'] == clusters_new[0][0]]
df = df.groupby(['time_date','degree','hostname'])['ip'].count().reindex().reset_index()
for node in clusters_new[1]:
    df_2 = df[df['hostname'] == node]
    df_2 = df_2.pivot_table(values='ip', index=df.time_date, columns='degree', aggfunc='first',fill_value=0).reset_index()
    display(df_2)
    df_2 = pd.melt(df_2,id_vars=['time_date'],value_vars=['주의','경고','심각'])
    #display(df_2)
    sns.factorplot(data=df_2, x="time_date", y="value",hue='degree',legend_out=True, size=3, aspect=4)
    plt.title(node)
    plt.xticks(rotation='vertical')

> #### 234 서버는 주말 관계없이 알람이 발생하는 하므로 서버 리소스 사용률이 평소에도 높은것으로 추측
> #### 231,236 서버는 주말에 알람이 거의 발생하지 않는 것으로 확인

### 11-2. 클러스터 0번에 속한 노드의 주요 지표 디스크 Top I/O 처리율에 대한 노드별 심각도 알람 건수 추이 출력
> (클러스터 0번이 첫번재 결과라 선택 함) - cluster0 ('HFEU-TREC234', 'HKCC-TREC231', 'HKCC-TREC236')
> 요일별 현황

In [ ]:
df = data[(data.alarm_name == '디스크 Top I/O 처리율') & (data['system_name'].isin(clusters_new[0]))]
#df = df[df['system_name'] == clusters_new[0][0]]
df = df.groupby(['time_weekday','degree','system_name'])['ip'].count().reindex().reset_index()
for node in clusters_new[0]:
    df_2 = df[df['system_name'] == node]
    df_2 = df_2.pivot_table(values='ip', index=df.time_weekday, columns='degree', aggfunc='first',fill_value=0).reset_index()
    df_2 = pd.melt(df_2,id_vars=['time_weekday'],value_vars=['주의','경고','심각'])
    display(df_2)
    sns.factorplot(data=df_2, x="time_weekday", y="value",hue='degree',legend_out=True, size=3, aspect=4)
    plt.title(node)

## 3그룹 - CPUs 사용률
### 심각 : 100% 이상
### 경고 : 90% 이상
### 주의 : 80% 이상

In [ ]:
clusters_new = ['']

In [ ]:
df = data[(data.alarm_name == 'TrafficRx') & (data['hostname'].isin(clusters_new))]
print(df)
condition  = df.condition.str.split(' ', expand=True)
print(condition)
df['type'] = condition[6]
df['threshold'] = condition[5].str.extract('(\d+(?:\.\d+)?)').astype(float)
df['current'] = condition[2].str.extract('(\d+(?:\.\d+)?)').astype(float)
print(df.threshold.unique())
df['current'] = np.where(condition[3]=='Gbps',df['current']*1000,df['current'])
print(df[df['threshold']==8][['threshold','degree','type']])
df_raw = df.groupby(['time','hostname','degree'])['current'].mean().reindex().reset_index()
df_hour = df.groupby(['time_date_hour','hostname','degree'])['current'].agg(['mean', 'min', 'max']).reindex().reset_index()
#display(df_hour)
for i,node in enumerate(clusters_new):
    print(node)
    df_2 = df_raw[df_raw['hostname'] == node]
    df_2 = df_2.pivot_table(values='current', index=df_2.time, columns='degree', aggfunc='first',fill_value=(np.NaN)).reset_index()
    df_2['merge'] = df_2.sum(axis=1)
    display(df_2)
    df_2.to_excel(str(i)+'.xlsx',encoding='UTF-8')
    df_3 = df_hour[df_hour['hostname'] == node]
    df_3.to_excel(str(i)+'_hour.xlsx',encoding='UTF-8')
    #df_2 = pd.melt(df_2,id_vars=['time'],value_vars=['주의','경고','심각'])
    #df_2['merge'] = df_2.sum(axis=1)
    #display(df_2)

 

In [ ]:
df = data[(data.alarm_name == 'CPUs 사용률') & (data['hostname'].isin(clusters_new))]
print(df.condition)
condition  = df.condition.str.split(' ', expand=True)
df['threshold'] = condition[4].str.extract('(\d+(?:\.\d+)?)').astype(float)
df['current'] = condition[1].str.extract('(\d+(?:\.\d+)?)').astype(float)
print(df.threshold.unique())
df_raw = df.groupby(['time','hostname','degree'])['current'].mean().reindex().reset_index()
df_hour = df.groupby(['time_date_hour','hostname','degree'])['current'].agg(['mean', 'min', 'max']).reindex().reset_index()
#display(df_hour)
for i,node in enumerate(clusters_new):
    print(node)
    df_2 = df_raw[df_raw['hostname'] == node]
    df_2 = df_2.pivot_table(values='current', index=df_2.time, columns='degree', aggfunc='first',fill_value=(np.NaN)).reset_index()
    df_2['merge'] = df_2.sum(axis=1)
    #display(df_2)
    df_2.to_excel(str(i)+'.xlsx',encoding='UTF-8')
    df_3 = df_hour[df_hour['hostname'] == node]
    df_3.to_excel(str(i)+'_hour.xlsx',encoding='UTF-8')
    #df_2 = pd.melt(df_2,id_vars=['time'],value_vars=['주의','경고','심각'])
    #df_2['merge'] = df_2.sum(axis=1)
    #display(df_2)

 

df = data[(data.alarm_name == 'CPUs 사용률') & (data['hostname'].isin(clusters_new))]
condition  = df.condition.str.split(' ', expand=True)
df['threshold'] = condition[4].str.extract('(\d+(?:\.\d+)?)').astype(float)
df['current'] = condition[1].str.extract('(\d+(?:\.\d+)?)').astype(float)
print(df.threshold.unique())
df_raw = df.groupby(['time','hostname','degree'])['current'].mean().reindex().reset_index()
df_hour = df.groupby(['time_date_hour','hostname','degree'])['current'].agg(['mean', 'min', 'max']).reindex().reset_index()
#display(df_hour)
for i,node in enumerate(clusters_new):
    print(node)
    df_2 = df_raw[df_raw['hostname'] == node]
    df_2 = df_2.pivot_table(values='current', index=df_2.time, columns='degree', aggfunc='first',fill_value=(np.NaN)).reset_index()
    df_2['merge'] = df_2.sum(axis=1)
    df_2 = pd.melt(df_2,id_vars=['time'],value_vars=['주의','경고','심각'])
    display(df_2)
    #sns.distplot(df_2["merge"], bins=50 )
    sns.pointplot(x="time", y="value", hue="degree", data=df_2, scale=0.2)
    sns.pairplot(df_2, 
             diag_kind='auto',
             hue="degree", 
             palette='bright') # pastel, bright, deep, muted, colorblind, dark
    plt.show()
    plt.clf()
    #display(df_2)
    df_3 = df_hour[df_hour['hostname'] == node]
    #df_2 = pd.melt(df_2,id_vars=['time'],value_vars=['주의','경고','심각'])
    #df_2['merge'] = df_2.sum(axis=1)
    #display(df_2)

 

In [ ]:
clusters_new = ['']
df = data[(data.alarm_name == 'CPUs 사용률') & (data['hostname'].isin(clusters_new))]
condition  = df.condition.str.split(' ', expand=True)
df['threshold'] = condition[4].str.extract('(\d+(?:\.\d+)?)').astype(float)
df['current'] = condition[1].str.extract('(\d+(?:\.\d+)?)').astype(float)
print(df.threshold.unique())
df_raw = df.groupby(['time','hostname','degree'])['current'].mean().reindex().reset_index()
df_hour = df.groupby(['time_date_hour','hostname','degree'])['current'].agg(['mean', 'min', 'max']).reindex().reset_index()
#display(df_hour)
for node in clusters_new:
    print(node)
    df_2 = df_raw[df_raw['hostname'] == node]
    df_2 = df_2.pivot_table(values='current', index=df_2.time, columns='degree', aggfunc='first',fill_value=(np.NaN)).reset_index()
    df_2['merge'] = df_2.sum(axis=1)
    display(df_2)
    df_3 = df_hour[df_hour['hostname'] == node]
    display(df_3)
    #df_2 = pd.melt(df_2,id_vars=['time'],value_vars=['주의','경고','심각'])
    #df_2['merge'] = df_2.sum(axis=1)
    #display(df_2)

## 4그룹 - ErrorRx
#### 심각 : >  30,  3회  연속
#### 주의 : 범위    안   5     ~   30,    3회  연속)]

In [ ]:
clusters_new = ['']
df = data[(data.alarm_name == 'ErrorRx') & (data['hostname'].isin(clusters_new))]
condition  = df.condition.str.split(' ', expand=True)
#df['threshold'] = condition[1].str.extract('(\d+(?:\.\d+)?)').astype(float)
df['current'] = condition[2].str.extract('(\d+(?:\.\d+)?)').astype(float)
#df = df.groupby(['time','hostname','degree'])['current'].mean().reindex().reset_index()
#display(df)
for node in clusters_new:
    print(node)
    df_2 = df[df['hostname'] == node]
    df_2 = df_2.pivot_table(values='current', index=df_2.time, columns='degree', aggfunc='first',fill_value=(np.NaN)).reset_index()
    display(df_2)
    print(df_2.describe())
    df_2 = pd.melt(df_2,id_vars=['time'],value_vars=['주의','경고','심각'])
    #display(df_2)

## 5그룹 - CPUs 사용률
### 심각 : 100% 이상
### 경고 : 90% 이상
### 주의 : 80% 이상

In [ ]:
clusters_new = ['']
df = data[(data.alarm_name == 'CPUs 사용률') & (data['hostname'].isin(clusters_new))]
condition  = df.condition.str.split(' ', expand=True)
df['threshold'] = condition[4].str.extract('(\d+(?:\.\d+)?)').astype(float)
df['current'] = condition[1].str.extract('(\d+(?:\.\d+)?)').astype(float)
print(df.threshold.unique())
df = df.groupby(['time','hostname','degree'])['current'].mean().reindex().reset_index()
for node in clusters_new:
    print(node)
    df_2 = df[df['hostname'] == node]
    df_2 = df_2.pivot_table(values='current', index=df_2.time, columns='degree', aggfunc='first',fill_value=(np.NaN)).reset_index()
    display(df_2)
    print(df_2.describe())
    df_2 = pd.melt(df_2,id_vars=['time'],value_vars=['주의','경고','심각'])
    #display(df_2)

## 6그룹 - CPUs 사용률
### 심각 : 100% 이상
### 경고 : 90% 이상
### 주의 : 80% 이상
## CPU Wait Process 수
### 경고 : 20 이상
### 주의 : 15 이상

In [ ]:
clusters_new = ['']
df = data[(data.alarm_name == 'CPUs 사용률') & (data['hostname'].isin(clusters_new))]
condition  = df.condition.str.split(' ', expand=True)
df['threshold'] = condition[4].str.extract('(\d+(?:\.\d+)?)').astype(float)
df['current'] = condition[1].str.extract('(\d+(?:\.\d+)?)').astype(float)
print(df.threshold.unique())
df = df.groupby(['time','hostname','degree'])['current'].mean().reindex().reset_index()
for node in clusters_new:
    print(node)
    df_2 = df[df['hostname'] == node]
    df_2 = df_2.pivot_table(values='current', index=df_2.time, columns='degree', aggfunc='first',fill_value=(np.NaN)).reset_index()
    display(df_2)
    print(df_2.describe())
    df_2 = pd.melt(df_2,id_vars=['time'],value_vars=['주의','경고','심각'])
    #display(df_2)

In [ ]:
df = data[(data.alarm_name == 'CPU Wait Process 수') & (data['hostname'].isin(clusters_new))]
condition  = df.condition.str.split(' ', expand=True)
df['threshold'] = condition[5].str.extract('(\d+(?:\.\d+)?)').astype(float)
df['current'] = condition[3].str.extract('(\d+(?:\.\d+)?)').astype(float)
print(df.threshold.unique())
df = df.groupby(['time','hostname','degree'])['current'].mean().reindex().reset_index()
for node in clusters_new:
    print(node)
    df_2 = df[df['hostname'] == node]
    df_2 = df_2.pivot_table(values='current', index=df_2.time, columns='degree', aggfunc='first',fill_value=(np.NaN)).reset_index()
    display(df_2)
    print(df_2.describe())
    df_2 = pd.melt(df_2,id_vars=['time'],value_vars=['주의','경고','심각'])
    #display(df_2)

## 7그룹 - CPUs 사용률
### 심각 : 100% 이상
### 경고 : 90% 이상
### 주의 : 80% 이상
## CPU Wait Process 수
### 경고 : 20 이상
### 주의 : 15 이상

In [ ]:
clusters_new = ['']
df = data[(data.alarm_name == 'CPUs 사용률') & (data['hostname'].isin(clusters_new))]
condition  = df.condition.str.split(' ', expand=True)
df['threshold'] = condition[4].str.extract('(\d+(?:\.\d+)?)').astype(float)
df['current'] = condition[1].str.extract('(\d+(?:\.\d+)?)').astype(float)
print(df.threshold.unique())
df = df.groupby(['time','hostname','degree'])['current'].mean().reindex().reset_index()
for node in clusters_new:
    print(node)
    df_2 = df[df['hostname'] == node]
    df_2 = df_2.pivot_table(values='current', index=df_2.time, columns='degree', aggfunc='first',fill_value=(np.NaN)).reset_index()
    display(df_2)
    print(df_2.describe())
    df_2 = pd.melt(df_2,id_vars=['time'],value_vars=['주의','경고','심각'])
    #display(df_2)

In [ ]:
df = data[(data.alarm_name == 'CPU Wait Process 수') & (data['hostname'].isin(clusters_new))]
condition  = df.condition.str.split(' ', expand=True)
df['threshold'] = condition[5].str.extract('(\d+(?:\.\d+)?)').astype(float)
df['current'] = condition[3].str.extract('(\d+(?:\.\d+)?)').astype(float)
print(df.threshold.unique())
df = df.groupby(['time','hostname','degree'])['current'].mean().reindex().reset_index()
for node in clusters_new:
    print(node)
    df_2 = df[df['hostname'] == node]
    df_2 = df_2.pivot_table(values='current', index=df_2.time, columns='degree', aggfunc='first',fill_value=(np.NaN)).reset_index()
    display(df_2)
    print(df_2.describe())
    df_2 = pd.melt(df_2,id_vars=['time'],value_vars=['주의','경고','심각'])
    #display(df_2)

## 10그룹 - CPUs 사용률
### 경고 : 90% 이상
### 주의 : 80% 이상

In [ ]:
clusters_new = ['']
df = data[(data.alarm_name == 'CPUs 사용률') & (data['hostname'].isin(clusters_new))]
condition  = df.condition.str.split(' ', expand=True)
df['threshold'] = condition[4].str.extract('(\d+(?:\.\d+)?)').astype(float)
df['current'] = condition[1].str.extract('(\d+(?:\.\d+)?)').astype(float)
print(df.threshold.unique())
df = df.groupby(['time','hostname','degree'])['current'].mean().reindex().reset_index()
for node in clusters_new:
    print(node)
    df_2 = df[df['hostname'] == node]
    df_2 = df_2.pivot_table(values='current', index=df_2.time, columns='degree', aggfunc='first',fill_value=(np.NaN)).reset_index()
    display(df_2)
    print(df_2.describe())
    df_2 = pd.melt(df_2,id_vars=['time'],value_vars=['주의','경고','심각'])
    #display(df_2)

## 11그룹 - CPUs 사용률
### 심각 : 100% 이상
### 경고 : 90% 이상
### 주의 : 80% 이상
## CPU Wait Process 수
### 경고 : 20 이상
### 주의 : 15 이상

In [ ]:
clusters_new = ['[S2][에] 차세대 영업 WAS #1','[S2][에] 차세대 영업 WAS #2']
df = data[(data.alarm_name == 'CPUs 사용률') & (data['hostname'].isin(clusters_new))]
condition  = df.condition.str.split(' ', expand=True)
df['threshold'] = condition[4].str.extract('(\d+(?:\.\d+)?)').astype(float)
df['current'] = condition[1].str.extract('(\d+(?:\.\d+)?)').astype(float)
print(df.threshold.unique())
df = df.groupby(['time','hostname','degree'])['current'].mean().reindex().reset_index()
for node in clusters_new:
    print(node)
    df_2 = df[df['hostname'] == node]
    df_2 = df_2.pivot_table(values='current', index=df_2.time, columns='degree', aggfunc='first',fill_value=(np.NaN)).reset_index()
    display(df_2)
    print(df_2.describe())
    df_2 = pd.melt(df_2,id_vars=['time'],value_vars=['주의','경고','심각'])
    #display(df_2)

In [ ]:
df = data[(data.alarm_name == 'CPU Wait Process 수') & (data['hostname'].isin(clusters_new))]
condition  = df.condition.str.split(' ', expand=True)
df['threshold'] = condition[5].str.extract('(\d+(?:\.\d+)?)').astype(float)
df['current'] = condition[3].str.extract('(\d+(?:\.\d+)?)').astype(float)
print(df.threshold.unique())
df = df.groupby(['time','hostname','degree'])['current'].mean().reindex().reset_index()
for node in clusters_new:
    print(node)
    df_2 = df[df['hostname'] == node]
    df_2 = df_2.pivot_table(values='current', index=df_2.time, columns='degree', aggfunc='first',fill_value=(np.NaN)).reset_index()
    display(df_2)
    print(df_2.describe())
    df_2 = pd.melt(df_2,id_vars=['time'],value_vars=['주의','경고','심각'])
    #display(df_2)

### 일자별로 알람 발생 당시의 성능 값 추이

In [ ]:
del data

In [ ]:
data_log.alarm_name.unique()

In [ ]:
#전체 'PLog Info','PLog Warn','PLog Error','PLog Fatal''AMS_ERROR_POS','AMS_ERROR''Log Warning-Error'
len(data_log)

In [ ]:
lists = ['PLog Info','PLog Warn','PLog Error','PLog Fatal']
for i in lists:
    data_plog = data_log[data_log.alarm_name==i]
    print('{} : {}'.format(i,len(data_plog)))
    data_plog['alarm_name'] = data_plog.condition.str.split(',',expand=True)[1].str.split('\[PLOG\]',expand=True)[1]
    data_plog.groupby(['alarm_name','hostname'])['ip'].count().reset_index().reindex().sort_values('ip',ascending=False).to_excel(i+'.xlsx',encoding='UTF-8')
    

In [ ]:
data_log[(data_log.alarm_name=='PLog Fatal')&(data_log.hostname=='[S1][백] 통합정보 APP #1 (LDOM Guest)')].groupby(['condition'])['ip'].count().reset_index().reindex().sort_values('ip',ascending=False)

In [ ]:
data_log[(data_log.alarm_name=='PLog Fatal')&(data_log.hostname=='[S1][백,Ex] 운영정보 DB #1')].groupby(['condition'])['ip'].count().reset_index().reindex().sort_values('ip',ascending=False)

In [ ]:
lists = ['AMS_ERROR_POS','AMS_ERROR']
for i in lists:
    data_ams = data_log[data_log.alarm_name==i]
    print('{} : {}'.format(i,len(data_ams)))
    data_ams['alarm_name'] = data_ams.condition.str.split('|',expand=True)[6]
    data_ams['alarm_type'] = data_ams.condition.str.split('|',expand=True)[7]
    data_ams.groupby(['alarm_name','alarm_type','hostname'])['ip'].count().reset_index().reindex().sort_values('ip',ascending=False).to_excel(i+'.xlsx',encoding='UTF-8')
    

## Log Warning-Error

In [ ]:
#'Log Info','Log Warning-Error'
data_logs = data_log[data_log.alarm_name=='Log Warning-Error']

In [ ]:
len(data_logs)

In [ ]:
data_logs_back = data_logs

In [ ]:
test = data_logs.condition.str.split('|',expand=True)[[7,8]]

In [ ]:
len(test[test[7].isnull()==False])

In [ ]:
len(data_logs)

In [ ]:
test[test[8].isnull()==False][7] = test[test[8].isnull()==False][8]

In [ ]:
test2 = np.where(test[8].isnull(),test[7],test[8])

In [ ]:
data_logs['alarm_type']= test2

In [ ]:
data_logs[data_logs.alarm_type.isnull()==False].groupby(['alarm_type'])['ip'].count().reset_index().reindex().sort_values('ip',ascending=False).to_excel('log_result01.xlsx',encoding='UTF-8')

In [ ]:
data_logs[data_logs.alarm_type.isnull()==False].groupby(['hostname','alarm_type'])['ip'].count().reset_index().reindex().sort_values('ip',ascending=False).to_excel('log_result.xlsx',encoding='UTF-8')

## 로그 데이터 형태소 분석

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=1000, stop_words='english')
tdm = cv.fit_transform(data_logs['condition'])
tdm.shape

In [ ]:
doc = tdm[0].toarray()
words = cv.get_feature_names()
[w
for w,c
in zip(words, doc.flat)
if c > 0]
count = tdm.sum(axis=0)

In [ ]:
word_count = list(zip(words, count.flat))
import operator

word_order = sorted(
    word_count,
    key=operator.itemgetter(1),
    reverse=True
)

In [ ]:
word_order[:1000]

In [ ]:
data_logs['condition'].str.split('|',expand=True)[7].isnull()==False)

In [ ]:
data_ams = data_log[data_log.alarm_name=='AMS_ERROR_POS']

In [ ]:
data_ams2 = data_log[data_log.alarm_name=='AMS_ERROR']

In [ ]:
data_ams[data_ams.ip=='10.253.32.119']

In [ ]:
data_ams2[data_ams2.ip=='10.253.32.119']

## Oracle Error

In [ ]:
len(data_logs[data_logs.condition.str.contains('ORA-')==True])

In [ ]:
data_logs_ora = data_logs[data_logs.condition.str.contains('ORA-')==True]
data_logs_ora['ora_code']= data_logs_ora.condition.str.extract(r'[ORA-](\d\d\d\d\d|\d\d\d\d|\d\d\d|\d\d|\d)')

In [ ]:
data_logs_ora.groupby(['hostname','ora_code'])['ip'].count().reset_index().reindex().sort_values('ip',ascending=False)

## TNS Error

In [ ]:
len(data_logs[data_logs.condition.str.contains('TNS-')==True])

In [ ]:
data_logs_tns = data_logs[data_logs.condition.str.contains('TNS-')==True]
data_logs_tns['tns_code']= data_logs_tns.condition.str.extract(r'[TNS-](\d\d\d\d\d|\d\d\d\d|\d\d\d|\d\d|\d)')
data_logs_tns.groupby(['hostname','tns_code'])['ip'].count().reset_index().reindex().sort_values('ip',ascending=False)

## DBLink

In [ ]:
len(data_logs[data_logs.condition.str.contains('DBLink:DC')==True])

In [ ]:
data_logs[data_logs.condition.str.contains('DBLink:DC')==True].groupby(['hostname'])['ip'].count().reset_index().reindex().sort_values('ip',ascending=False)

In [ ]:
data_logs[data_logs.condition.str.contains('DBLink:DC')==True].condition.str.extract(r'(SVR2)|(SVR1)')

In [ ]:
data_logs[data_logs.condition.str.contains('DBLink:DC')==True].condition

## JDBC Connection

In [ ]:
len(data_logs[data_logs.condition.str.contains('JDBC Connection')==True])

In [ ]:
data_logs_jdbc = data_logs[data_logs.condition.str.contains('JDBC Connection')==True]

### 1) OOM

In [ ]:
oom = data_logs_jdbc[data_logs_jdbc.condition.str.contains('OutOfMemoryError')==True]
oom['alarm_type'] = "OutOfMemoryError"
oom.groupby(['hostname','alarm_type'])['ip'].count().reset_index().reindex().sort_values('ip',ascending=False)

### 2) Could not open JDBC Connection

In [ ]:
connection = data_logs_jdbc[data_logs_jdbc.condition.str.contains('Could not open JDBC Connection')==True]
connection['alarm_type'] = "Could not open JDBC Connection"
connection.groupby(['hostname','alarm_type'])['ip'].count().reset_index().reindex().sort_values('ip',ascending=False)

## kernel

In [ ]:
len(data_logs[data_logs.condition.str.contains('kernel')==True])

In [ ]:
data_logs[data_logs.condition.str.contains('kernel')==True].groupby(['hostname'])['ip'].count().reset_index().reindex().sort_values('ip',ascending=False)

In [ ]:
len(data_logs[(data_logs.condition.str.contains('kernel')==True)&(data_logs.condition.str.contains('nfs')==True)])